2016-10-20 일 뉴스에서 하나의 뉴스와 비슷한 뉴스를 찾아봅니다.

In [1]:
import config
import sklearn
from navernews_10days import get_bow, get_news_paths
from soynlp.utils import DoublespaceLineCorpus

x, idx_to_vocab, vocab_to_idx = get_bow(date='2016-10-20', tokenize='noun')
path = get_news_paths(date='2016-10-20')
docs = [doc for doc in DoublespaceLineCorpus(path, iter_sent=False)]

print(sklearn.__version__)
print(x.shape)
print(len(docs))

soynlp=0.0.492
added lovit_textmining_dataset
0.20.3
(30091, 9774)
30091


임의의 문서 하나를 선택하였습니다. 도널드 트럼프 vs 힐러리 클린턴의 미국 대선에 대한 뉴스입니다.

이 뉴스의 term frequency vector 는 x[4,:] 으로 선택할 수 있습니다. 

In [2]:
query_idx = 15
docs[query_idx]

'클린턴 득표율 50 목표 가능성 아주 크다  워싱턴 연합뉴스 강영두 특파원 미국 민주당 대선후보 힐러리 클린턴이 다음달 8일 대선에서 400명 이상의 선거인단을 확보하면서 대승을 거둘 수 있다는 분석이 나왔다  미국 민주당 대선후보 힐러리 클린턴이 지난 10일 현지시간 오하이오 주 콜럼버스에서 유세하는 모습 연합뉴스  미 공화당 전략가인 스티브 슈미트는 19일 현지시간 방송의 모닝조 프로그램 인터뷰에서 이 같이 전망했다  슈미트는 2008년 존 매케인의 대선캠프에 관여했다  간접선거인 미 대선에서 승리에 필요한 선거인단 매직 넘버 는 전체 538명 의 과반인 270명이다  슈미트는 현 상황은 클린턴이 선거인단 400명을 넘기는 추세 라고 분석했다  그는 대선과 함께 치러지는 연방 상 하원의원 선거에 대해서도 상원은 민주당이 장악할 것으로 보이며 하원도 민주당이 장악할 수 있는 쪽으로 근접해 가고 있다 고 관측했다  현재는 공화당이 상원과 하원을 모두 장악하고 있다  슈미트는 만약 선거가 오늘 열린다고 가정하면 공화당은 하원에서 25석을 잃게 될 것으로 보인다 면서 공화당 입장에서는 선거판의 흐름을 보여주는 추세선이 잘못된 방향으로 가고 있다 내부에서 패닉이 시작됐다 고 말했다  미 일간 워싱턴포스트 가 서베이몽키와 15개 경합주에 대한 공동 여론조사 10월8 16일 1만7천379명 를 실시해 전날 발표한 결과를 보면 클린턴은 9개 주에서 공화당의 도널드 트럼프를 앞섰고 선거인단 304명을 확보한 것으로 나타났다  공화당 주변에서조차 트럼프의 완패 가능성이 거론되는 가운데 클린턴 진영은 득표율 50 고지 점령을 새로운 목표로 내걸었다  클린턴 측은 트럼프가 음담패설 비디오 파문과 성추문 논란에 휩싸이면 사실상 지지율을 견인할 추동력을 잃은 것으로 판단하고 있다 다만 자유당 게리 존슨 등 제3 후보 의 지지율이 10 안팎으로 상대적으로 높다는 점이 변수다  클린턴 캠프의 중량감 있는 한 인사는 의회전문매체 더힐 과의 인터뷰에서 득표율 50 전망에 대해 가능성이 

앞서 연습하였던 LSHForest 를 이용하여 Cosine distance 기준으로 유사한 문서를 찾겠습니다. training 은 lsh.fit(x) 로 끝납니다. 패러메터는 기본값을 이용하였습니다.

LSHForest 는 32 차원의 고정된 크기의 hash code 를 이용한다고 합니다. 우리는 그 외의 값들만 조절할 수 있습니다.

In [3]:
%%time

from sklearn.neighbors import LSHForest

lsh = LSHForest()
lsh.fit(x)

/home/lovit/anaconda3/envs/pytorch/lib/python3.7/site-packages/sklearn/neighbors/approximate.py:258: DeprecationWarning: LSHForest has poor performance and has been deprecated in 0.19. It will be removed in version 0.21.
  DeprecationWarning)


CPU times: user 768 ms, sys: 24 ms, total: 792 ms
Wall time: 788 ms


15 번 문서와 유사한 10개의 문서를 선택하였습니다. 가장 비슷한 문서는 15 번 이어야 합니다. 

In [4]:
from sklearn.metrics import pairwise_distances

query_vec = x[query_idx,:].reshape(1,-1)

In [5]:
%%time

dist, idxs = lsh.kneighbors(query_vec, n_neighbors=10)
print(idxs)

[[   15 15006 18139 24457 29966  6906 12549 16940 27167  3289]]
CPU times: user 40 ms, sys: 0 ns, total: 40 ms
Wall time: 36.8 ms


15 번을 포함한 10 개의 문서에 대하여 각 문서 별 200 글자까지만 snippest를 출력하여 그 내용을 확인하겠습니다. 이와 같이 비슷한 문서를 찾는 문제에서는 LSHForest를 쓰면 편리합니다. 

또한, 문서는 어느 정도 길이가 긴 문서인 것이 좋습니다. 짧은 문서는 too much sparse하여 인덱싱이 잘 되지 않을 수 있습니다. 

In [7]:
for d, idx in zip(dist[0], idxs[0]):
    print('[idx = {}, cos = {:.3}] {} ...\n'.format(idx, 1 - d, docs[idx][:200]))

[idx = 15, cos = 1.0] 클린턴 득표율 50 목표 가능성 아주 크다  워싱턴 연합뉴스 강영두 특파원 미국 민주당 대선후보 힐러리 클린턴이 다음달 8일 대선에서 400명 이상의 선거인단을 확보하면서 대승을 거둘 수 있다는 분석이 나왔다  미국 민주당 대선후보 힐러리 클린턴이 지난 10일 현지시간 오하이오 주 콜럼버스에서 유세하는 모습 연합뉴스  미 공화당 전략가인 스티브 슈미트는 1 ...

[idx = 15006, cos = 0.846] 미국 민주당 대선후보 힐러리 클린턴이 다음달 8일 대선에서 400명 이상의 선거인단을 확보하면서 대승을 거둘 수 있다는 분석이 나왔다  미 공화당 전략가인 스티브 슈미트는 현지시간 19일 방송의 모닝조 프로그램 인터뷰에서 이 같이 전망했다 슈미트는 2008년 존 매케인의 대선캠프에 관여했다 간접선거인 미 대선에서 승리에 필요한 선거인단 매직 넘버 는 전체  ...

[idx = 18139, cos = 0.505] 19일 현지 시간 미국 라스베이거스 네바다대학에서 열린 대선후보 3차 토론을 마친 후 민주당 대선후보 힐러리 클린턴이 공화당 대선후보 도널드 트럼프를 지나 청중에게 다가가고 있다 연합뉴스  미국 공화당 대선후보인 도널드 트럼프가 선거 결과에 불복할 수 있음을 강하게 내비치면서 대선판이 최악의 진흙탕 싸움으로 빠져들고 있다 선거 불복 논란이 그동안의 모든 선 ...

[idx = 24457, cos = 0.476] 3차 토론회에서 힐러리 클린턴 민주당 대선후보가 청중들에게 손을 흔들어 보이고 있다 연합뉴스 아시아경제 이지은 기자 이번 토론회의 승자도 힐러리 클린턴이었다  방송은 여론조사 기관 와 공동으로 19일 현지시간 네바다주 라스베이거스에서 열린 제 3차 토론회의 승자에 대해 여론조사를 실시한 결과 52 가 클린턴을 승자로 꼽았다고 보도했다 공화당 대선후보 도널드 ...

[idx = 29966, cos = 0.458] 미 대선 3차 토론회 여론조사 결과 52 대 39 클린턴 또 승리  미국 대선 레이

모든 데이터와의 거리를 직접 계산해 봅니다. 속도가 크게 차이가 나지 않아보입니다. 이는 탐색에 이용한 문서의 개수가 3 만 여개이기 때문입니다.

In [8]:
%%time

def full_search(query_vec, x, n_neighbors):
    dist = pairwise_distances(query_vec, x, metric='cosine')[0]
    sim_idxs = dist.argsort()[:n_neighbors]
    sim_dist = dist[sim_idxs]
    return sim_dist, sim_idxs

print(idxs)
dist, idxs = full_search(query_vec, x, n_neighbors=10)
print(idxs)

[[   15 15006 18139 24457 29966  6906 12549 16940 27167  3289]]
[   15 15006  7421 11309 13752 14182 10422 12536  3687  9194]
CPU times: user 660 ms, sys: 28 ms, total: 688 ms
Wall time: 229 ms


LSHForest 는 pickling 을 통하여 저장할 수도 있습니다. 결과는 학습했던 lsh 와 동일함을 확인할 수 있습니다. 하지만 데이터가 통째로 모델에 저장되어 있기 때문에 크기가 큽니다.

In [9]:
import pickle

with open('test_lsh.pkl', 'wb') as f:
    pickle.dump(lsh, f)
    
with open('test_lsh.pkl', 'rb') as f:
    loaded_lsh = pickle.load(f)
    
_, idxs = loaded_lsh.kneighbors(query_vec, n_neighbors=10)
idxs

array([[   15, 15006, 18139, 24457, 29966,  6906, 12549, 16940, 27167,
         3289]])

LSHForest 는 scikit-learn 의 0.21 부터 제공되지 않는다고 합니다. 그 외에는 Ball Tree 나 KD Tree 가 이용되지만, Ball Tree 와 KD Tree 는 sparse matrix 를 지원하지 않습니다 (scikit=0.20.3 기준). 또한 Ball Tree 와 KD-Tree 는 cosine 거리를 제공하지 않습니다. 두 인덱서가 제공하는 거리 척도는 아래처럼 확인할 수 있습니다.

```python
from sklearn.neighbors import BallTree, KDTree

print(BallTree.valid_metrics)
print(KDTree.valid_metrics)
```

모든 벡터의 L2 norm 이 1 일 경우 (unit vector), 벡터 간 Euclidean distance 거리는 Cosine distance 의 2 배 입니다. 이 경우에는 벡터를 정규화 한 뒤, Euclidean distance 를 이용할 수 있습니다.

In [10]:
import time
import numpy as np
from sklearn.neighbors import BallTree
from sklearn.neighbors import LSHForest

def comparision(x):

    # training time comparison
    lsh_train_time = time.time()
    lsh = LSHForest()
    lsh.fit(x)
    lsh_train_time = time.time() - lsh_train_time

    bt_train_time = time.time()
    ball_tree = BallTree(x, leaf_size=10, metric='euclidean')
    bt_train_time = time.time() - bt_train_time

    # query time & quality comparison
    query_vec = x[:10] # sample queries

    lsh_query_time = time.time()
    lsh_dist, idxs = lsh.kneighbors(query_vec, n_neighbors=10)
    lsh_query_time = time.time() - lsh_query_time

    bt_query_time = time.time()
    bt_dist, idxs = ball_tree.query(query_vec, k=10)
    bt_query_time = time.time() - bt_query_time

    # when all vectors are unit vector, 2 * cosine distance is equal euclidean distance
    # cosine dist = 1 - product # prodict = cosine sim
    # euclidean dist = 2 - 2 * product
    print('training time (LSH Forest) = {:.3} sec'.format(lsh_train_time))
    print('training time (Ball Tree) = {:.3} sec'.format(bt_train_time))
    print('query time (LSH Forest) = {:.3} sec, mean dist = {:.6}'.format(lsh_query_time, 2 * lsh_dist.mean()))
    print('query time (Ball Tree) = {:.3} sec, mean dist = {:.6}'.format(bt_query_time, bt_dist.mean()))

In [11]:
from sklearn.preprocessing import normalize

x = np.random.random_sample((100000, 100))
x = normalize(x, norm='l2')
comparision(x)

/home/lovit/anaconda3/envs/pytorch/lib/python3.7/site-packages/sklearn/neighbors/approximate.py:258: DeprecationWarning: LSHForest has poor performance and has been deprecated in 0.19. It will be removed in version 0.21.
  DeprecationWarning)


training time (LSH Forest) = 0.588 sec
training time (Ball Tree) = 0.572 sec
query time (LSH Forest) = 0.0387 sec, mean dist = 0.313637
query time (Ball Tree) = 0.275 sec, mean dist = 0.486448


In [12]:
x = np.random.random_sample((100000, 32))
x = normalize(x, norm='l2')
comparision(x)

/home/lovit/anaconda3/envs/pytorch/lib/python3.7/site-packages/sklearn/neighbors/approximate.py:258: DeprecationWarning: LSHForest has poor performance and has been deprecated in 0.19. It will be removed in version 0.21.
  DeprecationWarning)


training time (LSH Forest) = 0.402 sec
training time (Ball Tree) = 0.21 sec
query time (LSH Forest) = 0.0259 sec, mean dist = 0.170063
query time (Ball Tree) = 0.118 sec, mean dist = 0.358388
